In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [2]:
# Extracting music and features
'''
Dataset
We use GTZAN genre collection dataset for classification. 

The dataset consists of 10 genres i.e

Blues
Classical
Country
Disco
Hiphop
Jazz
Metal
Pop
Reggae
Rock
Each genre contains 100 songs. Total dataset: 1000 songs
'''

'\nDataset\nWe use GTZAN genre collection dataset for classification. \n\nThe dataset consists of 10 genres i.e\n\nBlues\nClassical\nCountry\nDisco\nHiphop\nJazz\nMetal\nPop\nReggae\nRock\nEach genre contains 100 songs. Total dataset: 1000 songs\n'

In [3]:
# Extracting the Spectrogram for every Audio

In [4]:
'''
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'img_data/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'img_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
'''

'\ncmap = plt.get_cmap(\'inferno\')\n\nplt.figure(figsize=(10,10))\ngenres = \'blues classical country disco hiphop jazz metal pop reggae rock\'.split()\nfor g in genres:\n    pathlib.Path(f\'img_data/{g}\').mkdir(parents=True, exist_ok=True)     \n    for filename in os.listdir(f\'./MIR/genres/{g}\'):\n        songname = f\'./MIR/genres/{g}/{filename}\'\n        y, sr = librosa.load(songname, mono=True, duration=5)\n        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides=\'default\', mode=\'default\', scale=\'dB\');\n        plt.axis(\'off\');\n        plt.savefig(f\'img_data/{g}/{filename[:-3].replace(".", "")}.png\')\n        plt.clf()\n'

In [5]:
'''
Extracting features from Spectrogram
We will extract

Mel-frequency cepstral coefficients (MFCC)(20 in number)
Spectral Centroid,
Zero Crossing Rate
Chroma Frequencies
Spectral Roll-off.
'''

'\nExtracting features from Spectrogram\nWe will extract\n\nMel-frequency cepstral coefficients (MFCC)(20 in number)\nSpectral Centroid,\nZero Crossing Rate\nChroma Frequencies\nSpectral Roll-off.\n'

In [6]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [7]:
# Writing data to csv file

In [9]:
'''
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./MIR/genres/{g}'):
        songname = f'./MIR/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
'''

"\nfile = open('data.csv', 'w', newline='')\nwith file:\n    writer = csv.writer(file)\n    writer.writerow(header)\ngenres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()\nfor g in genres:\n    for filename in os.listdir(f'./MIR/genres/{g}'):\n        songname = f'./MIR/genres/{g}/{filename}'\n        y, sr = librosa.load(songname, mono=True, duration=30)\n        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)\n        rmse = librosa.feature.rmse(y=y)\n        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)\n        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)\n        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)\n        zcr = librosa.feature.zero_crossing_rate(y)\n        mfcc = librosa.feature.mfcc(y=y, sr=sr)\n        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    \n        for e in mfcc:\n            to_append += f' {np.

In [10]:
# Analysing the Data in Pandas

In [11]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00060.au,0.430894,0.196222,1946.565652,1979.909934,3955.867746,0.097454,-67.770980,111.704184,-34.646105,...,12.295832,-12.477988,1.681278,-5.142068,4.644002,-6.919217,1.040718,-4.736871,-0.660037,blues
1,blues.00082.au,0.338896,0.251350,2141.461656,2168.015560,4627.997015,0.105151,-29.362093,108.667950,-25.573165,...,5.456504,-7.687713,7.410600,-11.319177,7.229288,-9.466552,1.930059,-6.328476,-1.304812,blues
2,blues.00030.au,0.263016,0.170081,1379.081742,2004.000850,3015.831764,0.039376,-206.987590,117.781468,23.256245,...,-8.015467,-17.616342,-8.138554,-8.646157,-15.538988,-15.331506,-9.664872,-10.103310,-17.835100,blues
3,blues.00007.au,0.307921,0.131785,1451.754147,1577.369917,2955.348796,0.061435,-179.395447,136.459244,-26.656359,...,-6.954827,-3.544535,-8.051242,-8.959537,-8.424337,-10.558885,-10.788159,-4.693749,-8.638613,blues
4,blues.00064.au,0.332480,0.117413,2553.232415,2280.128669,5148.102203,0.146852,-85.150250,88.806722,-16.322611,...,8.478453,-19.590226,6.413210,-13.779667,6.112037,-13.154644,3.933456,-7.615454,3.752626,blues


In [12]:
data.shape

(1000, 28)

In [13]:
# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)

In [14]:
# Encoding the Labels

In [15]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [16]:
# Scaling the Feature columns

In [17]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [18]:
# Dividing data into training and Testing set

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
len(y_train)

800

In [21]:
len(y_test)

200

In [22]:
X_train[10]

array([ 1.71595489, -0.63589658,  1.05952008,  0.59990028,  0.7811424 ,
        0.41284744, -0.76332485, -1.86542645, -0.23416499, -0.16058369,
        1.09112317, -0.08197994,  1.11531191, -0.23787937,  1.48397167,
        0.34567857,  1.58239438,  0.30198017,  1.83009669,  0.73451768,
        1.32239052, -0.60830427,  0.53152504,  0.66674309,  1.93340958,
        1.25442381])

In [23]:
# Classification with Keras
# Building our Network

In [24]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [25]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [26]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
800/800 [==============================] - 2s 2ms/step - loss: 2.1790 - acc: 0.2112
Epoch 2/20
800/800 [==============================] - 0s 83us/step - loss: 1.8650 - acc: 0.3887
Epoch 3/20
800/800 [==============================] - 0s 100us/step - loss: 1.6385 - acc: 0.4262
Epoch 4/20
800/800 [==============================] - 0s 91us/step - loss: 1.4634 - acc: 0.5125
Epoch 5/20
800/800 [==============================] - 0s 100us/step - loss: 1.3121 - acc: 0.5550
Epoch 6/20
800/800 [==============================] - 0s 77us/step - loss: 1.1986 - acc: 0.6275
Epoch 7/20
800/800 [==============================] - 0s 81us/step - loss: 1.1067 - acc: 0.6438
Epoch 8/20
800/800 [==============================] - 0s 79us/step - loss: 1.0210 - acc: 0.6687
Epoch 9/20
800/800 [==============================] - 0s 110us/step - loss: 0.9571 - acc: 0.6875
Epoch 10/20
800/800 [==============================] - 0s 85us/step - loss: 0.8972 - acc: 0.7075
Epoch 11/20
800/800 [================

In [27]:
test_loss, test_acc = model.evaluate(X_test,y_test)

200/200 [==============================] - 0s 745us/step


In [28]:
print('test_acc: ',test_acc)

test_acc:  0.7


In [29]:
# Tes accuracy is less than training data accuracy. This hints at Overfitting.

In [30]:
# Validating our approach
# Let's set apart 200 samples in our training data to use as a validation set:

In [31]:
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [32]:
# Now let's train our network for 20 epochs:

In [33]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Train on 600 samples, validate on 200 samples
Epoch 1/30
600/600 [==============================] - 1s 2ms/step - loss: 2.3223 - acc: 0.0733 - val_loss: 2.1589 - val_acc: 0.2950
Epoch 2/30
600/600 [==============================] - 0s 119us/step - loss: 2.1226 - acc: 0.3517 - val_loss: 2.0320 - val_acc: 0.3300
Epoch 3/30
600/600 [==============================] - 0s 112us/step - loss: 1.9711 - acc: 0.3633 - val_loss: 1.9225 - val_acc: 0.3150
Epoch 4/30
600/600 [==============================] - 0s 110us/step - loss: 1.8310 - acc: 0.3733 - val_loss: 1.8215 - val_acc: 0.3400
Epoch 5/30
600/600 [==============================] - 0s 115us/step - loss: 1.7097 - acc: 0.3967 - val_loss: 1.7299 - val_acc: 0.4050
Epoch 6/30
600/600 [==============================] - 0s 108us/step - loss: 1.5954 - acc: 0.4600 - val_loss: 1.6492 - val_acc: 0.4400
Epoch 7/30
600/600 [==============================] - 0s 118us/step - loss: 1.4869 - acc: 0.5050 - val_loss: 1.5757 - val_acc: 0.4400
Epoch 8/30
600/600

In [34]:
results

[1.0170895957946777, 0.70999999999999996]

In [35]:
# Predictions on Test Data

In [36]:
predictions = model.predict(X_test)

In [37]:
predictions[0].shape

(10,)

In [38]:
np.sum(predictions[0])

1.0

In [39]:
np.argmax(predictions[0])

7